""""

1. Tried pulling directly from the website source api, got my ass kicked
2. Skywalk API is working and they fixed some requests
3. Testing dashboard combined with Apartments.com data-puller

""""

In [1]:
import requests
from bs4 import BeautifulSoup
import json

appfolio_api_key = "wc9DeJ9UHoJKkEsgr9CN4W.8AnBp5DgrPE2HNdNMQ1HxwDlBkJnYRZDyGvletEznkE"
payload = {"X-API-KEY" : appfolio_api_key}
params = { "search": "?propertyId=51"}
r = requests.get('https://api.skywalkapi.com/v1/balance-sheet', headers=payload, params = params)
r.raise_for_status()

In [2]:
json.loads(r.content)

{'meta': {'status': 'updating',
  'path': '/v1/balance-sheet',
  'lastUpdated': None,
  'jobId': 'ff3a9408-efb8-4e87-81ce-a2c1a8718a27',
  'query': {'accounting': 'cash', 'date': '02/15/2022'}}}

# Dashboard

In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time

import ApartmentsScraperHelper

bb = "xtqrw9_u9Isz8svW"
# pull the same code everytime but for San Marco
ApartmentsScraperHelper.main(bb = bb)

pd.read_csv(bb+".csv")

NameError: name 'csv' is not defined

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html
import pandas as pd

#zillow rent data
df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv?t=1644973146")
df.drop(columns= ["RegionID", "SizeRank"], inplace=True)
df.head()

,RegionName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,2014-07,2014-08,2014-09,...,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01
0,United States,1294.0,1299,1305.0,1310,1315,1321,1326,1331,1337,...,1708,1729,1750,1771,1793,1815.0,1837.0,1859,1881,1904
1,"New York, NY",2256.0,2266,2276.0,2286,2296,2306,2316,2326,2336,...,2526,2556,2586,2617,2648,2680.0,2712.0,2746,2780,2814
2,"Los Angeles-Long Beach-Anaheim, CA",1792.0,1803,1814.0,1825,1835,1846,1857,1868,1878,...,2497,2524,2550,2577,2604,2631.0,2659.0,2687,2716,2744
3,"Chicago, IL",1411.0,1415,1419.0,1423,1426,1430,1433,1437,1440,...,1649,1662,1675,1688,1702,1716.0,1729.0,1744,1758,1772
4,"Dallas-Fort Worth, TX",1128.0,1132,1135.0,1139,1143,1146,1150,1154,1157,...,1534,1556,1578,1600,1622,1645.0,1667.0,1690,1714,1737


In [14]:
reframe_df = pd.DataFrame(df.set_index("RegionName").stack()).reset_index()
reframe_df.columns = ["RegionName", "Date", "ZRent"]
reframe_df.RegionName = reframe_df.RegionName.str.replace(",", "")
reframe_df

,RegionName,Date,ZRent
0,United States,2014-01,1294.0
1,United States,2014-02,1299.0
2,United States,2014-03,1305.0
3,United States,2014-04,1310.0
4,United States,2014-05,1315.0
...,...,...,...
9880,College Station TX,2021-09,1524.0
9881,College Station TX,2021-10,1532.0
9882,College Station TX,2021-11,1541.0
9883,College Station TX,2021-12,1550.0


In [15]:
### Multiselect functionality
""""
NO COMMAS, messes up dropdown for some reason
""""

app = JupyterDash(__name__)
del app.config._read_only["requests_pathname_prefix"]

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reframe_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZRents'),

        dcc.Graph(
            id='Zrent',
        )]
                  )])
@app.callback(
    Output('Zrent', 'figure'),
    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reframe_df[reframe_df.RegionName.isin(selected_cities)]   
    fig = px.scatter(filtered_df, x="Date", y="ZRent", color="RegionName", hover_name="RegionName")

    fig.update_layout(transition_duration=500)

    return fig
    
if __name__ == '__main__':
    app.run_server(debug=False)

/Users/nelsonlin/Desktop/AYPOClicker/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

127.0.0.1 - - [15/Feb/2022 18:00:58] "GET /_shutdown_31d202e5-dd8e-44e4-bd31-85cf65100c86 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Feb/2022 18:00:59] "GET /_alive_31d202e5-dd8e-44e4-bd31-85cf65100c86 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8052/


127.0.0.1 - - [15/Feb/2022 18:01:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2022 18:01:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2022 18:01:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Feb/2022 18:01:00] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2022 18:01:00] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2022 18:01:00] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Feb/2022 18:01:00] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL']


127.0.0.1 - - [15/Feb/2022 18:01:03] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL', 'New York NY']


127.0.0.1 - - [15/Feb/2022 18:01:06] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL', 'New York NY', 'Philadelphia PA']


### Add apartment list too

In [2]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import Input, Output, dcc, html
import pandas as pd

#zillow rent data
df = pd.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_ZORI_AllHomesPlusMultifamily_SSA.csv?t=1644973146")
df.drop(columns= ["RegionID", "SizeRank"], inplace=True)

reframe_df = pd.DataFrame(df.set_index("RegionName").stack()).reset_index()
reframe_df.columns = ["RegionName", "Date", "ZRent"]
reframe_df.RegionName = reframe_df.RegionName.str.replace(",", "")

app = JupyterDash(__name__)
del app.config._read_only["requests_pathname_prefix"]

app.layout = html.Div([
    html.Div(children=[
        
        html.Br(),
        html.Label('Pick Cities to compare'),
        dcc.Dropdown(options = reframe_df.RegionName.unique(),
                     value = ["United States", "Jacksonville FL"],
                     multi=True, 
                     id="cities_to_graph"),
       
    ]),
    
    html.Div(children=[
        
        html.Br(),
        html.Label('Graph of ZRents'),

        dcc.Graph(
            id='Zrent',
        )]
                  )])
@app.callback(
    Output('Zrent', 'figure'),
    Input('cities_to_graph', 'value'))

def update_figure(selected_cities):
    
    print("selected_cities", selected_cities)
    if not selected_cities:
        selected_cities = ["United States"]
        
    filtered_df = reframe_df[reframe_df.RegionName.isin(selected_cities)]   
    fig = px.scatter(filtered_df, x="Date", y="ZRent", color="RegionName", hover_name="RegionName")

    fig.update_layout(transition_duration=500)

    return fig
    
if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Feb/2022 16:58:39] "GET /_alive_da31f8aa-6e51-4ee1-8bca-1aa3547d40bb HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [18/Feb/2022 16:58:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-component-suites/dash/deps/react@16.v2_1_0m1644801654.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_1_0m1644801654.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_1_0m1644801654.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_1_0m1644801654.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_1_0m1644801654.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_1_0m1644801654.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:50] "GET /_dash

selected_cities ['United States', 'Jacksonville FL']


127.0.0.1 - - [18/Feb/2022 16:58:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2022 16:58:54] "POST /_dash-update-component HTTP/1.1" 200 -


selected_cities ['United States', 'Jacksonville FL', 'Dallas-Fort Worth TX']
